In [1]:
pwd

'/home/wsuser/work'

In [2]:
!pip install keras
!pip install tensorflow

In [3]:
# This library helps add support for large, multi-dimensional arrays and matrices
import numpy as np
#open source used for both ML and DL for computation
import tensorflow as tf
#it is a plain stack of layers
from tensorflow.keras.models import Sequential 
#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Dense,Flatten, Dropout
#Faltten-used fot flattening the input or change the dimension, MaxPooling2D-for downsampling the image for Convolutional layer
from tensorflow.keras.layers import Convolution2D,MaxPooling2D 
#Its used for different augmentation of the image 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Augment the Data

In [4]:
#setting parameter for Image Data agumentation to the training data
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
#Image Data agumentation to the testing data
test_datagen=ImageDataGenerator(rescale=1./255)

# Loading our data and performing data agumentation

In [5]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_30e1afd35dff4763ae7c1098184fd72b = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='du6YA7C0pfJ4HyyBJgPxBMs4fDBUXPVt60o5wAKw57R4',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

streaming_body_2 = client_30e1afd35dff4763ae7c1098184fd72b.get_object(Bucket='gesturebasedtool-donotdelete-pr-ernxppvolkdsvt', Key='Dataset.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [6]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_2.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [7]:
ls -l

total 8
drwxrwx--- 8 wsuser wscommon 4096 Aug 17 16:30 test/
drwxrwx--- 8 wsuser wscommon 4096 Aug 17 16:30 train/


In [8]:
#performing data agumentation to train data
x_train = train_datagen.flow_from_directory(r'/home/wsuser/work/train',
                                            target_size=(64, 64),
                                            batch_size=3,
                                            color_mode='grayscale',
                                            class_mode='categorical')
#performing data agumentation to test data
x_test = test_datagen.flow_from_directory(r'/home/wsuser/work/test',
                                          target_size=(64, 64),
                                          batch_size=3,
                                          color_mode='grayscale',
                                          class_mode='categorical') 

Found 594 images belonging to 6 classes.
Found 30 images belonging to 6 classes.


In [9]:
print(x_train.class_indices)#checking the number of classes

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5}


# Model Creation

In [10]:
# Initializing the CNN
model = Sequential()

In [11]:
# First convolution layer and pooling
model.add(Convolution2D(32, (3, 3), input_shape=(64, 64, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [12]:
# Second convolution layer and pooling
model.add(Convolution2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
model.add(MaxPooling2D(pool_size=(2,2)))

In [13]:
# Flattening the layers i.e. input layer
model.add(Flatten())

In [14]:
# Adding a fully connected layer, i.e. Hidden Layer
model.add(Dense(units=512 , activation='relu'))

In [15]:
# softmax for categorical analysis, Output Layer
model.add(Dense(units=6, activation='softmax')) 

In [16]:
model.summary()#summary of our model

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 512)               3

# Modal compailation

In [17]:
# Compiling the CNN
# categorical_crossentropy for more than 2
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 

# Modal Fitting

In [18]:
# It will generate packets of train and test data for training
model.fit_generator(x_train,
                    steps_per_epoch = 594/3 , 
                    epochs = 25, 
                    validation_data = x_test,
                    validation_steps = 30/3 )

/tmp/wsuser/ipykernel_164/804983804.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(x_train,


Epoch 1/25
198/198 [==============================] - 8s 37ms/step - loss: 1.3511 - accuracy: 0.4680 - val_loss: 0.9062 - val_accuracy: 0.6000
Epoch 2/25
198/198 [==============================] - 7s 36ms/step - loss: 0.6112 - accuracy: 0.7508 - val_loss: 0.6769 - val_accuracy: 0.6667
Epoch 3/25
198/198 [==============================] - 7s 36ms/step - loss: 0.4335 - accuracy: 0.8232 - val_loss: 0.4465 - val_accuracy: 0.9000
Epoch 4/25
198/198 [==============================] - 7s 36ms/step - loss: 0.2901 - accuracy: 0.8990 - val_loss: 0.4318 - val_accuracy: 0.8667
Epoch 5/25
198/198 [==============================] - 7s 36ms/step - loss: 0.2061 - accuracy: 0.9411 - val_loss: 0.5768 - val_accuracy: 0.8667
Epoch 6/25
198/198 [==============================] - 7s 36ms/step - loss: 0.1789 - accuracy: 0.9394 - val_loss: 0.3476 - val_accuracy: 0.9000
Epoch 7/25
198/198 [==============================] - 7s 36ms/step - loss: 0.1710 - accuracy: 0.9259 - val_loss: 0.2741 - val_accuracy: 0.9000

# Saving model

In [19]:
# Save the model
model.save('gesture.h5')

In [20]:
model_json = model.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)

In [21]:
!tar -zcvf Gesture-model_new.tgz gesture.h5

gesture.h5


In [22]:
ls -l

total 71296
-rw-rw---- 1 wsuser wscommon 38740208 Aug 17 16:34 gesture.h5
-rw-rw---- 1 wsuser wscommon 34250057 Aug 17 16:34 Gesture-model_new.tgz
-rw-rw---- 1 wsuser wscommon     2823 Aug 17 16:34 model-bw.json
drwxrwx--- 8 wsuser wscommon     4096 Aug 17 16:30 test/
drwxrwx--- 8 wsuser wscommon     4096 Aug 17 16:30 train/


In [23]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 23.8 MB/s eta 0:00:01


In [24]:
from ibm_watson_machine_learning import APIClient
wml_credentials={
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey":"5MZ_SyRLQnnSw-u94L-p3XDEp3ELgQe24GOELm1ve0ju"
}
client=APIClient(wml_credentials)

In [25]:
client=APIClient(wml_credentials)

In [26]:
def guid_from_space_name(client,space_name):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"]==space_name)['metadata']['id'])

In [27]:
space_uid=guid_from_space_name(client,'Gesture_tool')
print("Space UID= "+space_uid)

Space UID= 5ee550a9-6181-4141-9b0e-5ce942febaf3


In [28]:
client.set.default_space(space_uid)

'SUCCESS'

In [29]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1-py3.9             12b83a1

In [30]:
import tensorflow as tf
tf.__version__

'2.7.2'

In [31]:
import keras as ks
ks.__version__

'2.7.0'

In [41]:
software_space_uid=client.software_specifications.get_uid_by_name('tensorflow_rt22.1-py3.9')

In [42]:
software_space_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [43]:
ls -l

total 71296
-rw-rw---- 1 wsuser wscommon 38740208 Aug 17 16:34 gesture.h5
-rw-rw---- 1 wsuser wscommon 34250057 Aug 17 16:34 Gesture-model_new.tgz
-rw-rw---- 1 wsuser wscommon     2823 Aug 17 16:34 model-bw.json
drwxrwx--- 8 wsuser wscommon     4096 Aug 17 16:30 test/
drwxrwx--- 8 wsuser wscommon     4096 Aug 17 16:30 train/


In [44]:
model_details=client.repository.store_model(model='Gesture-model_new.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN Model Building",
    client.repository.ModelMetaNames.TYPE:'tensorflow_2.7',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_uid
})

In [46]:
model_details

{'entity': {'hybrid_pipeline_software_specs': [],
  'software_spec': {'id': 'acd9c798-6974-5d2f-a657-ce06e986df4d',
   'name': 'tensorflow_rt22.1-py3.9'},
  'type': 'tensorflow_2.7'},
 'metadata': {'created_at': '2022-08-17T16:46:30.034Z',
  'id': '544654d1-c7b6-44d0-9c99-41166ff6e475',
  'modified_at': '2022-08-17T16:46:35.270Z',
  'name': 'CNN Model Building',
  'owner': 'IBMid-6660028MBA',
  'resource_key': 'b5aa7889-3855-4a8f-8bc8-ee35e5f20bfe',
  'space_id': '5ee550a9-6181-4141-9b0e-5ce942febaf3'},
 'system': {'warnings': []}}

In [47]:
model_id=client.repository.get_model_id(model_details)

In [48]:
model_id

'544654d1-c7b6-44d0-9c99-41166ff6e475'

In [50]:
client.repository.download(model_id,'Gesture.tar.gb')

Successfully saved model content to file: 'Gesture.tar.gb'


'/home/wsuser/work/Gesture.tar.gb'